In [16]:
using IterativeSolvers
using LinearAlgebra
using SparseArrays

# Define a sparse matrix A
n = 100
function sparse_matrix(N::Int, factor::Int)
    A = zeros(Float64, N, N)
    for i in 1:N
        for j in 1:N
            if i == j
                A[i, j] = rand() 
            else
                A[i, j] = rand() / factor
            end
        end
    end
    A = 0.5 * (A + A')

    return Hermitian(A)
    
end

A = sparse_matrix(n, 10)


100×100 Hermitian{Float64, Matrix{Float64}}:
 0.856881   0.0496213   0.0724674  …  0.0325698  0.0512905  0.0311582
 0.0496213  0.530585    0.0796147     0.0675785  0.0386912  0.0823244
 0.0724674  0.0796147   0.103092      0.0610791  0.0204155  0.0784671
 0.0629393  0.0382297   0.0621467     0.070809   0.0740243  0.0319316
 0.0645702  0.0609854   0.0287161     0.0576385  0.0850686  0.076689
 0.0354435  0.0500243   0.0476784  …  0.0735638  0.0211217  0.0515135
 0.0391039  0.0289119   0.049453      0.0745365  0.0296191  0.0304714
 0.0950648  0.0442384   0.0628653     0.06355    0.058309   0.0647815
 0.0631378  0.0784094   0.0346784     0.062456   0.0507987  0.0485495
 0.0468216  0.00902535  0.0818114     0.0396013  0.0323102  0.0320546
 ⋮                                 ⋱                        
 0.0405194  0.0424189   0.0358249     0.0124238  0.0620076  0.0475681
 0.0449497  0.0300588   0.0524489     0.0745701  0.0233758  0.0776992
 0.0772621  0.0590437   0.0680376     0.053137   0.0655

In [18]:
θ = 1.0  # Shift
I_mat = Matrix{Float64}(I, n, n)  # Identity matrix

# Define a unit vector u
u = randn(n)
u /= norm(u)

# Define a random right-hand side residual r
r = randn(n)

# Define the projection operator P = I - u*u'
function project(x)
    return x - u * (u' * x)
end

# Define the operator M = (I - uuᵗ)(A - θI)(I - uuᵗ)
function M_mul!(y, x)
    tmp = project(x)                      # (I - uuᵗ) * x
    tmp2 = (A - θ*I_mat) * tmp                # (A - θI)(I - uuᵗ)x
    y .= project(tmp2)                   # (I - uuᵗ)(A - θI)(I - uuᵗ)x
end

# Wrap into a LinearOperator
M_op = LinearOperator(Float64, n, n, false, false, M_mul!)

# Solve the system using BiCGStab(l)
l = 2
t_approx, history = bicgstabl(M_op, -r, l; log=true)

# Print results
println("Approximate solution t:\n", t_approx)
println("\nRelative residual norm: ", norm(project((A - θ*I) * project(t_approx)) + r) / norm(r))


Approximate solution t:
[101.5865954477878, 57.80652304574696, 66.09529096945374, -33.889672950646215, -33.603006386586706, -14.966713809004737, 24.646954225726596, -20.243413817681024, 153.79864617894867, 2.680854127668728, -82.22758928335868, -60.90316817688521, -62.542495464990324, -16.385483452802223, 13.301367666527803, 32.67204419239878, -103.3223321522256, 179.66555261826255, -68.03626582426787, 29.950309467218787, 1.0514176762986658, 30.95537978670997, -41.25444587272512, -79.9669266400965, -41.02621847981415, -55.65513820036037, 84.91812042891294, 25.44267230764117, 72.5339594552121, -61.78648042383117, 8.835158328754352, 26.20467986267513, 76.76018358364806, -36.60315183124401, 69.69667551774525, 145.39095309545314, 222.7507352425541, -81.8117882223684, -89.34028121829563, 51.36275884460835, -144.6986017327847, -116.38909017311812, -44.79944591812595, -29.907655734004017, -32.42188596695, -107.19831536023372, 5.361176804021245, -63.89496474309153, 33.60043865928893, 104.40211

In [2]:
using LinearAlgebra

function sparse_matrix(N::Int, factor::Int)
    A = zeros(Float64, N, N)
    for i in 1:N
        for j in 1:N
            if i == j
                A[i, j] = rand() 
            else
                A[i, j] = rand() / factor
            end
        end
    end
    A = 0.5 * (A + A')

    return Hermitian(A)
end

A = sparse_matrix(1000, 10)

@time begin
    # Compute the Schur decomposition
    schur_decomposition = schur(A)
end


schur_matrix = schur_decomposition.T
schur_vectors = schur_decomposition.Z

Eigenvalues = diag(schur_matrix)
println("Eigenvalues:")
println(Eigenvalues)

# exact eigenvalues
@time begin
    # Compute the eigenvalues using the eigen function
    eigen_decomposition = eigen(A)
end
#sorted_indices
# sorted_indices = sortperm(-exact_eigenvalues)
# exact_eigenvalues = exact_eigenvalues[sorted_indices]
exact_eigenvalues = eigen_decomposition.values
println("Exact Eigenvalues:")
println(exact_eigenvalues)



  0.424811 seconds (29 allocations: 30.877 MiB, 26.31% gc time)
Eigenvalues:
[-0.9400093172406433, -0.9311711635150159, -0.9174342339933101, -0.9103046858546763, -0.9006726734969107, -0.8953356079622943, -0.8877920406925712, -0.874703351253286, -0.8725535988981903, -0.8670026852990431, -0.8621454906016606, -0.8526244971729313, -0.8511439864223256, -0.8465895058686248, -0.8360180663560431, -0.8334932402335445, -0.8223056326284416, -0.8210758054657121, -0.8125064104660793, -0.8068773777702111, -0.8000769345251844, -0.7962479688841059, -0.7930277115740371, -0.7918032512135957, -0.7843764532196646, -0.7809814892313177, -0.7778159269186009, -0.7734851784676879, -0.7683735364816107, -0.7656712270984202, -0.763988460483926, -0.7592776619830831, -0.7568683066200806, -0.7515973748946799, -0.7446392688110326, -0.7373828518110299, -0.7348971201889749, -0.7322048215034755, -0.7310408619267934, -0.7259268625564212, -0.7196873511716699, -0.716016461837094, -0.7104165389195843, -0.7091035071550716, -

In [1]:
y = [1, 2, 3, 4, 5]
println(y)

[1, 2, 3, 4, 5]


In [2]:
println("Hello, Julia!")

Hello, Julia!


In [3]:
println(collect(1:1:10))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [4]:
for i in collect(1:3:30)
    println(i)
end

1
4
7
10
13
16
19
22
25
28


In [5]:
println(rand())


0.04696119735283866


In [6]:

function sparse_matrix(N::Int, factor::Int)
    A = zeros(Float64, N, N)
    for i in 1:N
        for j in 1:N
            if i == j
                A[i, j] = rand() * factor
            else
                if rand() < 0.05 # 5% chance to be non-zero off-diagonal --> Problemmm
                    A[i, j] = rand() / factor
                else
                    # Keep the off-diagonal elements as zero
                    A[i, j] = 0.0
                end
            end
        end
    end

    return Hermitian(A)
end

sparse_matrix (generic function with 1 method)

In [7]:
A = sparse_matrix(10, 50)

UndefVarError: UndefVarError: `Hermitian` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LinearAlgebra in the current active module Main

In [8]:
diag(A)

UndefVarError: UndefVarError: `diag` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LinearAlgebra in the current active module Main

In [9]:
using Plots
using LinearAlgebra

N = 2000
factor = 10
γ_pp = sparse_matrix(N, factor)

# Replace small values with a threshold
for i = 1:N
    for j = 1:N
        if abs(γ_pp[i,j] <= 1e-6)
            γ_pp[i,j] = 1e-6
        end
    end
end

logabs_γ_pp = log10.(abs.(γ_pp))
heatmap(logabs_γ_pp,
        xlabel="b",
        ylabel="a",
        colorbar_title="color map",
        aspect_ratio=:equal,
        c = :ice,
        size = (800, 800))


ArgumentError: ArgumentError: Cannot set a non-diagonal index in a Hermitian matrix